## 各种import

In [1]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("../examples"))

import utils; reload(utils)
from utils import *

from __future__ import division, print_function

Using TensorFlow backend.


## 从Keras导入数据

In [2]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## 处理数据

### One-Hot-Encoding

In [3]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道

In [4]:
trn_data = np.expand_dims(trn_data, 1)
test_data = np.expand_dims(test_data, 1)
trn_data.shape

(60000, 1, 28, 28)

### 对输入图像进行正则化

In [5]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

## 1. 简单神经网络模型

In [6]:
simpleNN = Sequential()
simpleNN.add(Lambda(norm_input, input_shape=(1, 28, 28)))
simpleNN.add(Flatten())
simpleNN.add(Dense(512, activation='softmax'))
simpleNN.add(Dense(10, activation='softmax'))
# 这里要把optimizer=Adam() 改成为 Adam()才会比较好, 为什么?
simpleNN.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

### 1.1 直接训练

In [9]:
#simpleNN.optimizer.lr = 0.1
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 17s - loss: 1.2175 - acc: 0.8959 - val_loss: 0.6028 - val_acc: 0.9289


In [8]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 16s - loss: 1.1765 - acc: 0.5399 - val_loss: 1.1229 - val_acc: 0.5472
Epoch 2/3
60000/60000 [==============================] - 16s - loss: 1.1632 - acc: 0.5477 - val_loss: 1.1065 - val_acc: 0.5672
Epoch 3/3
60000/60000 [==============================] - 15s - loss: 1.1738 - acc: 0.5511 - val_loss: 1.1937 - val_acc: 0.5570


### 1.2 生成batch再训练

利用ImageDataGenerator(), 使用fit_generator而不是fit
注意fit_generator的文档中参数由于版本升级发生了变化

In [7]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

In [9]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 18s - loss: 0.5881 - acc: 0.8114 - val_loss: 0.5546 - val_acc: 0.8269


In [11]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.5534 - acc: 0.8212 - val_loss: 0.5068 - val_acc: 0.8295


## 2. 类似vgg16的卷积神经网络模型

In [22]:
simpleCNN = Sequential([
    (Lambda(norm_input, input_shape=(1, 28, 28), output_shape=(1, 28, 28))),
    (Convolution2D(32, 3, 3, activation="relu")),
    (Convolution2D(32, 3, 3, activation="relu")),
    (MaxPooling2D()),
    (Convolution2D(64, 3, 3, activation="relu")),
    (Convolution2D(64, 3, 3, activation="relu")),
    (MaxPooling2D()),
    (Flatten()),
    (Dense(1024, activation="softmax")),
    (Dense(10, activation="softmax"))])
simpleCNN.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
simpleCNN.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_11 (Lambda)               (None, 1, 28, 28)     0           lambda_input_11[0][0]            
____________________________________________________________________________________________________
convolution2d_45 (Convolution2D) (None, 32, 26, 26)    320         lambda_11[0][0]                  
____________________________________________________________________________________________________
convolution2d_46 (Convolution2D) (None, 32, 24, 24)    9248        convolution2d_45[0][0]           
____________________________________________________________________________________________________
maxpooling2d_24 (MaxPooling2D)   (None, 32, 12, 12)    0           convolution2d_46[0][0]           
___________________________________________________________________________________________

In [ ]:
simpleCNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
15072/60000 [======>.......................] - ETA: 333s - loss: 1.9815 - acc: 0.5441

In [ ]:
simpleCNN.optimizer.lr = 0.1
simpleCNN.fit_generator(batches, 60000, nb_epoch=1, validation_data=test_batches, nb_val_samples=10000)

In [6]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
model = get_model()

In [9]:
model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 456s - loss: 0.0990 - acc: 0.9688 - val_loss: 0.0325 - val_acc: 0.9899
